# Цель
Используя нейронные сети определить уйдёт клиент из банка или осстанется

# Импорты

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2022-07-02 08:36:03.739422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-02 08:36:03.740019: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 1. Подготовка данных

In [33]:
df = pd.read_csv('./Churn_Modelling.csv').drop('RowNumber', axis=1)
X = df.iloc[:, 2:-1].values # Не нужны колонки CustomerId, Surname, CreditScore и Exited. Убирём их для тренировочных данных
y = df.iloc[:, -1].values # нужен только Exited

In [34]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [35]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [36]:
df.describe(include='all')

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1.000000e+04,10000,10000.000000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
unique,NaN,2932,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Smith,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,32,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.569094e+07,NaN,650.528800,NaN,NaN,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,NaN,96.653299,NaN,NaN,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,NaN,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,NaN,584.000000,NaN,NaN,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,NaN,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,NaN,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000


## Енкодинг
Машина понимает только числа, поэтому мы переведём строки в числовые версии, к примеру 1-мужчина, 0-девушка.

In [37]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
X[:, 2] = l.fit_transform(X[:, 2])

In [38]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [40]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

## Делим данные

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature Scaling
Изначально мы предпологаем что все наши параметры одинакого важны, но машина думает что если возраст 41, а пол(1-мужчина, 0-девушка), то возраст в разы важнее.

Нужно перевести все параметры к одному диапазону, это и называется feature scaling.

Не буду вдаваться в подробности работы этого алгоритма на старте

In [42]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [43]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# 2. Постройка нейронной сети(ANN)
Ипользется keras, это библиотека внутри tensorflow, которая помогает проще работать с ней

In [44]:
ann = tf.keras.models.Sequential() #инициализируем нейронную сеть

## Слои нейронной сети
У keras есть такая фишка, что на самом деле мы определяем второй слой. Входной слой данных, где появляются параметры, определяется сам.

In [45]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # второй слой нейронов, состоящий из 6 нейронов.
# активатоор relu - график, который равен нулю до определённого значения, а потом резко становится равен еденице

In [46]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #третий слой нейронов

In [47]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# сигмоидный активатор часто используют для выходных слоёв, потому что у неё есть вероятность. То есть, 0.52 это скорее 1 чем 0


# 3. Обучение
У нас появась архитектура нейронной сети, теперь можно обучить её
## Компиляция

In [48]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# loss - считает число ошибок
# optimizer - оптимизирует градиентный спуск
# metrics - метрики

## Тренировка нейронной сети

In [49]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)
# Мы указали 100 эпох(epochs=100), то есть нейронка пройдёт 100 раз через весь датасэт

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5850 - accuracy: 0.7268
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4693 - accuracy: 0.8075
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4374 - accuracy: 0.8175
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4229 - accuracy: 0.8200
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4152 - accuracy: 0.8253
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4081 - accuracy: 0.8306
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3998 - accuracy: 0.8328
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3903 - accuracy: 0.8384
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3805 - accuracy: 0.8434
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3718 - accura

# 4. Предсказание

Представим клиента с такими характеристиками:
Geography: France,
Credit Score: 600,
Gender: Male,
Age: 40 years,
Tenure: 3 years,
Balance: $6000,
Number of Products: 2,
Does this customer have a credit card? Yes,
Is this customer an Active Member: Yes,
Estimated salary: $5000,

In [51]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# По поводу этого клиента можно не бояться, он не уйдёт

1/1 [==============================] - 1s 749ms/step
[[False]]


## Проверяем точность предсказания

In [52]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 1s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Делаем Confusion Matrix

In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1532   63]
 [ 213  192]]


0.862